In [ ]:
import os
import glob
import cv2
import shutil
data_dir='Data'+os.sep+'FPS change'

In [ ]:
videos_list=glob.glob(os.path.join(data_dir,'**','*.MOV'),recursive=True)+glob.glob(os.path.join(data_dir,'**','*.MP4'),recursive=True)

In [ ]:
def extract_srt_data(srt_path):
    with open(srt_path, "r") as srt:
        data = []
        lines = srt.readlines()
        lines_count = 0   
        file_done = False 
        frame_counter=0
           
        while not file_done and lines_count < len(lines):
            group = [] # Each element of this list represent details of indvidual Frame
            frame_done = False
            frame_lines_counter = 0
            
            while not frame_done and lines_count < len(lines):
                line = lines[lines_count]

                if line.strip() == "" or frame_lines_counter > 7 : # means end of a frame
                    frame_counter+=1 
                    frame_done = True
                    lines_count += 1  
                    if frame_counter%2 == 1:
                        data.append(group)       
                    break
                else:
                    group.append(line)
                    frame_lines_counter += 1
                    lines_count += 1
                    
            if not group:
                # Skip empty groups
                continue
                    
            
            if lines_count >= len(lines):
                file_done = True
                break      
    return data

In [ ]:
  
def frames2video(input_folder, output_video_path, fps=30):
    """
    Convert a sequence of images in a folder to a video.

    Args:
        input_folder (str): Path to the folder containing image frames.
        output_video_path (str): Path to the output video file.
        fps (int, optional): Frames per second for the output video. Default is 30.

    Returns:
        None

    Example:
        input_folder_path = 'path/to/your/image/folder'
        output_video_path = 'output_video.mp4'
        images_to_video(input_folder_path, output_video_path)
    """
    # Get the list of image files in the input folder
    image_files = [f for f in os.listdir(input_folder) if f.endswith('.jpg')]
    image_files.sort()  # Sort the files in ascending order

    # Determine the width and height from the first image
    img = cv2.imread(os.path.join(input_folder, image_files[0]))
    height, width, _ = img.shape
    print('before delete: ',output_video_path)

    if os.path.exists(output_video_path):
        os.remove(output_video_path)
        
    print('after delete: ',output_video_path)
    # Create a video writer object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # You can change the codec based on your preference
    video_writer = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    # Iterate through the image files and write each frame to the video
    for image_file in image_files:
        img_path = os.path.join(input_folder, image_file)
        img = cv2.imread(img_path)
        video_writer.write(img)

    # Release the video writer
    video_writer.release()    
    
    # Check if the folder exists
    picture_extensions = ['jpg', 'jpeg', 'png', 'gif', 'bmp']
    picture_pattern = os.path.join(input_folder, f'*.[{";".join(picture_extensions)}]')
    picture_files = glob.glob(picture_pattern)
    for picture_file in picture_files:
        try:
            os.remove(picture_file)
        except Exception as e:
            print(f"Error deleting {picture_file}: {e}")
    shutil.rmtree(input_folder)

In [ ]:
for that_vid in videos_list:
    vid_dir = os.path.dirname(that_vid)
    srt_file_path = glob.glob(vid_dir + os.sep + '*.SRT')[0]
    
    new_vidpath=vid_dir+os.sep+'newVideo.MP4'
    frames_path=os.path.join(vid_dir, "Frames" + os.sep)
    
    cap = cv2.VideoCapture(that_vid)
    frames_per_second = cap.get(cv2.CAP_PROP_FPS)
    Total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT) - 1)
    srtData=extract_srt_data(srt_file_path)
    
    srtnewpath=vid_dir+os.sep+'newsrt.srt'
    with open(srtnewpath, "w") as file:
        for items in srtData:
            for eachitem in items:
                file.write(f"{eachitem}")
            file.write(f"\n")

    frame_counter=0
    
    while frame_counter < Total_frames:
        frame_counter += 1
        ret, frame = cap.read()
        if not ret:
            break  # Break the loop if the video is over
        if frame_counter%2 == 1:
            frame_path = os.path.join(that_vid.split("DJI_")[0], "Frames" + os.sep)

            if not os.path.exists(frame_path):
                os.makedirs(frame_path)

            cv2.imwrite(f"{frame_path}result_{frame_counter:09d}.jpg", frame)
            
    frames2video(frames_path, new_vidpath, 30)